In [1]:
# NAME PARAMS
drive_folder = "drive/My Drive/[NN] Music Generation/"    # workspace folder in gdrive
dataset_path = "JazzDataset/"                             # dataset path
channel = "Piano"                                         # midi channel to extract

name = "test_name"                                        # model name
checkPoint_path = './checkpoint/weights_'+name+'.hdf5'    # path to save model
log_dir = './logs'                                        # path for Tensorboard logs

In [16]:
# TRAINING PARAMS
vocab_threshold = 50
seq_length = 64
batch_size = 32
n_epochs = 50
test_ratio = 0.05

# NET PARAMS
hidden_units = 300
dropout_rate = 0.1
dense_units = 256
optimizer_lstm = "nadam"
loss_lstm = "categorical_crossentropy"

#Drive Access

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir(drive_folder)

# Libraries

In [5]:
from preprocessing import *
from models import lstmNet
from postprocessing import *
from sklearn.model_selection import train_test_split
from keras.callbacks import *

# Implementation

In [ ]:
print("Loading MID dataset...")
mid_songs = load_dataset(dataset_path, channel)

Loading MID dataset...
Loading music from:  JazzDataset/Duke _ His Orchestra Ellington - Satin Doll 2.mid
Loading music from:  JazzDataset/Louis Armstrong - Sweet Adeline.mid
Loading music from:  JazzDataset/Ray_Charles_-_What_d_I_Say.mid
Loading music from:  JazzDataset/Fats Waller - Wish Upon A Star.mid
Loading music from:  JazzDataset/Ray_Charles_-_Georgia_On_My_Mind.mid
Loading music from:  JazzDataset/Reinhardt Django - After You`ve Gone.mid
Loading music from:  JazzDataset/Glenn _ His Orchestra Miller - Chattanooga Choo Choo 2.mid
Loading music from:  JazzDataset/Jazz Standardsdiversen - Lullaby Of Birdland.mid
Loading music from:  JazzDataset/Fitzgerald_Ella_Ray_Johnny_-_All_Of_Me.mid
Loading music from:  JazzDataset/Fats Waller - Tea For Two.mid
Loading music from:  JazzDataset/Duke _ His Orchestra Ellington - The Mooch.mid
Loading music from:  JazzDataset/Louis Armstrong - C`Est Si Bon.mid
Loading music from:  JazzDataset/Louis Prima - Sing Sing Sing.mid
Loading music from:  J

In [6]:
print("Loading CSV dataset...")
csv_songs = load_csv_dataset("./Jazz-midi.csv")
print("Extracted "+str(len(csv_songs))+" songs")

Loading CSV dataset...
Extracted 818 songs


In [10]:
#songs = mid_songs #+ csv_songs[0:400] # for full dataset
#songs = mid_songs # only for JazzSet
songs = csv_songs[0:50]

In [12]:
print("Creating vocabularies...")
vocab, inverted_vocab, counts = make_vocabs(songs, vocab_threshold)
print("Vocabs size: "+str(len(vocab)))

Creating vocabularies...
Vocabs size: 108


In [13]:
print("Dataset preprocessing...")
x, y = dataset_preprocessing(songs, vocab, seq_length)
print("Dataset Shape:")
print("|->X: "+str(x.shape))
print("|->Y: "+str(y.shape))

Dataset preprocessing...
Dataset Shape:
|->X: (36757, 64, 1)
|->Y: (36757, 108)


In [14]:
print("Splitting dataset...")
train_x, dev_x, train_y, dev_y = train_test_split(x, y, test_size=test_ratio)
print("Training-Set Shape:")
print(train_x.shape)
print(train_y.shape)
print("Dev-Set Shape:")
print(dev_x.shape)
print(dev_y.shape)

Splitting dataset...
Training-Set Shape:
(34919, 64, 1)
(34919, 108)
Dev-Set Shape:
(1838, 64, 1)
(1838, 108)


In [17]:
output_size = len(vocab)
n_lstm = lstmNet(seq_length, hidden_units, dropout_rate, dense_units, output_size, optimizer_lstm, loss_lstm)
n_lstm.summary()

(None, 64, 1)
(None, 64, 300)
(None, 64, 300)
(None, 300)
(None, 300)
(None, 256)
(None, 256)
(None, 108)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64, 300)           362400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               77056     
______________

In [ ]:
callbacks = [ModelCheckpoint(filepath = checkPoint_path, save_weights_only=True, monitor='loss', mode='min', save_best_only=True), TensorBoard(log_dir=log_dir) ]
history = n_lstm.fit(train_x, train_y, batch_size=batch_size, epochs = n_epochs, validation_data=(dev_x, dev_y), callbacks = callbacks, shuffle=True)

In [ ]:
model = n_lstm.load_weights(checkPoint_path)
gen_song = generate_song(nn_wavenet, inverted_vocab, 100, train_x, seq_length)
print("Generated song:\n",gen_song)
create_midi(gen_song, "Piano", "", name+".mid")
print("Song saved on drive.")